# Libraries and dataset and path location

In [ ]:
import numpy as np
import librosa
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import joblib
from scipy import stats

from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os
import librosa
import numpy as np
import joblib


import joblib
import pickle
from joblib import dump

# !pip install pydub

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
import os
import librosa
import numpy as np
import joblib
from scipy import stats

from sklearn.ensemble import VotingClassifier, BaggingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib

In [ ]:
# # this code is used for splitting the audio in 1 sec each

# from pydub import AudioSegment
# import os

# # Set the duration of each segment to 1 second
# segment_duration = 1000  # in milliseconds

# # Create a directory for the segmented audio files
# output_dir = "/content/noise_full"
# os.makedirs(output_dir, exist_ok=True)

# # Get a list of all WAV files in the input directory
# input_dir = "/content/drive/MyDrive/_noise/_noise"
# wav_files = [f for f in os.listdir(input_dir) if f.endswith('.wav')]

# # Variable to keep track of the total number of segments processed
# total_segments = 0

# # Loop over each WAV file and segment it
# for wav_file in wav_files:
#     # Load the audio file using a raw string literal
#     input_path = os.path.join(input_dir, wav_file)
#     audio = AudioSegment.from_file(input_path, format="wav")

#     # Divide the audio file into segments
#     segments = []
#     for i in range(0, len(audio), segment_duration):
#         segment = audio[i:i + segment_duration]
#         segments.append(segment)

#     # Check if the last segment is shorter than the desired duration
#     if len(segments[-1]) < segment_duration:
#         # If the last segment is shorter, remove it from the segments list
#         segments.pop()

#     # Export each segment to the output directory
#     for i, segment in enumerate(segments):
#         output_path = os.path.join(output_dir, f"{os.path.splitext('nnoise_')[0]}_{total_segments + i + 1}.wav")
#         segment.export(output_path, format="wav")

#     # Update the total number of segments processed
#     total_segments += len(segments)

# print("Segmentation and saving completed successfully!")

In [ ]:
##this code is used1 to count number of audio files in a folder
# import os

# # Function to recursively count audio files
# def count_audio_files(folder_path):
#     count = 0
#     for root, dirs, files in os.walk(folder_path):
#         for file in files:
#             if file.endswith('.wav'):
#                 count += 1
#     return count

# # Specify the folder path
# folder_path = '/content/drive/MyDrive/new_training_dataset/noise_full'

# # Call the function to count audio files
# num_audio_files = count_audio_files(folder_path)

# # Print the number of audio files
# print(f"Number of audio files in the folder: {num_audio_files}")


# Path Location and other global access code variable

In [ ]:
# Set the paths to your audio data folders
drone_folder = '/content/drive/MyDrive/new_training_dataset/drone'
swarm_drone_folder = '/content/drive/MyDrive/new_training_dataset/Copy_of_swarm'
noise_folder = '/content/drive/MyDrive/new_training_dataset/noise_44100'

In [ ]:
# Define the number of files to take from each folder
num_files = 4361

# Mfcc for window size = 1024, audio = 1 sec and performing grid search and hyperparameter tuning

In [ ]:
# Define the window size for MFCC (in samples)
window_size = 1024

# Define the number of files to consider from each folder
num_files = 4361

# Initialize empty lists for features and labels
features = []
labels = []

# Load MFCC features and assign labels for noise class
counter = 0
for filename in os.listdir(noise_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(noise_folder, filename)
        audio, sr = librosa.load(file_path)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2)
        features.append(mfcc.flatten())
        labels.append(0)
        counter += 1

# Load MFCC features and assign labels for drone class
counter = 0
for filename in os.listdir(drone_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(drone_folder, filename)
        audio, sr = librosa.load(file_path)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2)
        features.append(mfcc.flatten())
        labels.append(1)
        counter += 1

# Load MFCC features and assign labels for swarm_drone class
counter = 0
for filename in os.listdir(swarm_drone_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(swarm_drone_folder, filename)
        audio, sr = librosa.load(file_path)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2)
        features.append(mfcc.flatten())
        labels.append(2)
        counter += 1

# Convert features and labels to numpy arrays
X = np.array(features)
y = np.array(labels)

# Perform normalization on the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter grid for grid search
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'degree': [2, 3, 4],
    'gamma': ['scale', 'auto']
}

# Create the SVM model
svm_model = SVC()

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5)

# Fit the model to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters: ", best_params)
print("Best Score: {:.2f}%".format(best_score*100))

# Create the SVM model with the best hyperparameters
svm_model = SVC(**best_params)

# Train the model on the training data
svm_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = svm_model.predict(X_test)

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Define class labels
class_labels = ['Noise', 'Drone', 'Swarm Drone']

# Print the confusion matrix in count of files
print("Confusion Matrix:")
for i, label in enumerate(class_labels):
    print("{:<12s}".format(label), end="")
    for j in range(len(class_labels)):
        print("{:6d}".format(conf_matrix[i, j]), end=" ")
    print("   True: {}".format(label))

# Display the column labels
print("\nPredicted:  ", end="")
for label in class_labels:
    print("{:<12s}".format(label), end=" ")
print()

# Save the trained model
model_filename = "best_svm_model_mfcc_one_sec.joblib"
dump(svm_model, model_filename)
print("Model saved as", model_filename)

Best Parameters:  {'C': 10, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
Best Score: 97.23%
Accuracy: 97.52%
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.95      0.97       897
           1       0.97      0.99      0.98       852
           2       0.97      0.99      0.98       868

    accuracy                           0.98      2617
   macro avg       0.98      0.98      0.98      2617
weighted avg       0.98      0.98      0.98      2617

Confusion Matrix:
Noise          852     20     25    True: Noise
Drone            5    845      2    True: Drone
Swarm Drone     11      2    855    True: Swarm Drone

Predicted:  Noise        Drone        Swarm Drone  
Model saved as best_svm_model_mfcc_one_sec.joblib


# Mfcc window size = 1024, audio = [0.6, 0.8, 1.0] second and run svm based on hyperparameter tuning

In [ ]:
# Define the target durations in seconds
target_durations = [0.6, 0.8, 1.0]

# Define the number of files to take from each folder
num_files = 4361

# Define the window size and hop size for MFCC extraction
window_size = 1024
hop_size = 512

# Initialize empty lists for features and labels
all_features = []
all_labels = []

for duration in target_durations:
    features = []
    labels = []

    # Load features and assign labels for noise class
    counter = 0
    for filename in os.listdir(noise_folder):
        if filename.endswith('.wav'):
            file_path = os.path.join(noise_folder, filename)
            audio, sr = librosa.load(file_path, duration=duration)

            mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_fft=window_size, hop_length=hop_size)
            features.append(mfcc.flatten())
            labels.append(0)
            counter += 1

            if counter >= num_files:
                break

    # Load features and assign labels for drone class
    counter = 0
    for filename in os.listdir(drone_folder):
        if filename.endswith('.wav'):
            file_path = os.path.join(drone_folder, filename)
            audio, sr = librosa.load(file_path, duration=duration)

            mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_fft=window_size, hop_length=hop_size)
            features.append(mfcc.flatten())
            labels.append(1)
            counter += 1

            if counter >= num_files:
                break

    # Load features and assign labels for swarm_drone class
    counter = 0
    for filename in os.listdir(swarm_drone_folder):
        if filename.endswith('.wav'):
            file_path = os.path.join(swarm_drone_folder, filename)
            audio, sr = librosa.load(file_path, duration=duration)

            mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_fft=window_size, hop_length=hop_size)
            features.append(mfcc.flatten())
            labels.append(2)
            counter += 1

            if counter >= num_files:
                break

    # Convert features and labels to numpy arrays
    X = np.array(features)
    y = np.array(labels)

    # Perform normalization on the features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the SVM model with the best hyperparameters
    svm_model = SVC(C=10, kernel='rbf', degree=2, gamma='scale')

    # Train the model on your training data
    svm_model.fit(X_train, y_train)

    # Save the SVM model
    model_name = "{:.1f}s_mfcc_svm_model.joblib".format(duration)
    joblib.dump(svm_model, model_name)
    print("Saved model:", model_name)

    # Make predictions on your test data
    y_pred = svm_model.predict(X_test)

    # Calculate the accuracy score
    accuracy = accuracy_score(y_test, y_pred)
    print("Duration: {:.1f}s".format(duration))
    print("Accuracy: {:.2f}%".format(accuracy*100))

    # Print the classification report
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print()

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Define class labels
    class_labels = ['Noise', 'Drone', 'Swarm Drone']

    # Print the confusion matrix in count of files
    print("Confusion Matrix:")
    for i, label in enumerate(class_labels):
        print("{:<12s}".format(label), end="")
        for j in range(len(class_labels)):
            print("{:6d}".format(conf_matrix[i, j]), end=" ")
        print("   True: {}".format(label))

    # Display the column labels
    print("\nPredicted:  ", end="")
    for label in class_labels:
        print("{:<12s}".format(label), end=" ")
    print()
    # Add line markers
    print('\n' + '-'*40 + '\n')




Saved model: 0.6s_mfcc_svm_model.joblib
Duration: 0.6s
Accuracy: 97.44%
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.95      0.97       897
           1       0.97      0.99      0.98       852
           2       0.97      0.98      0.97       868

    accuracy                           0.97      2617
   macro avg       0.97      0.97      0.97      2617
weighted avg       0.97      0.97      0.97      2617


Confusion Matrix:
Noise          850     21     26    True: Noise
Drone            2    847      3    True: Drone
Swarm Drone     12      3    853    True: Swarm Drone

Predicted:  Noise        Drone        Swarm Drone  

----------------------------------------

Saved model: 0.8s_mfcc_svm_model.joblib
Duration: 0.8s
Accuracy: 97.59%
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.95      0.97       897
           1       0.97      1.00      0.98       852
 

# Mel spec win len = 1024, perform grid search and best hyperparameter

In [ ]:
# Define the number of files to take from each folder
num_files = 4361
# Define the window size for Mel spectrogram (in samples)
window_size = 1024

features = []
labels = []

# Load Mel spectrogram features and assign labels for noise class
counter = 0
for filename in os.listdir(noise_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(noise_folder, filename)
        audio, sr = librosa.load(file_path, duration=1.0)

        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2)
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec_features = mel_spec.flatten()
        features.append(mel_spec_features)
        labels.append(0)
        counter += 1

# Load Mel spectrogram features and assign labels for drone class
counter = 0
for filename in os.listdir(drone_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(drone_folder, filename)
        audio, sr = librosa.load(file_path, duration=1.0)

        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2)
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec_features = mel_spec.flatten()
        features.append(mel_spec_features)
        labels.append(1)
        counter += 1

# Load Mel spectrogram features and assign labels for swarm_drone class
counter = 0
for filename in os.listdir(swarm_drone_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(swarm_drone_folder, filename)
        audio, sr = librosa.load(file_path, duration=1.0)

        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2)
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec_features = mel_spec.flatten()
        features.append(mel_spec_features)
        labels.append(2)
        counter += 1

# Convert features and labels to numpy arrays
X = np.array(features)
y = np.array(labels)

# Perform normalization on the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the SVM classifier
svm = SVC()

# Define the parameter grid for grid search
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(svm, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and train the model
best_svm = grid_search.best_estimator_
best_svm.fit(X_train, y_train)

# Save the best SVM model
model_name = "1s_melspectrogram_best_svm_model.joblib"
joblib.dump(best_svm, model_name)
print("Saved model:", model_name)

# Evaluate the model
y_pred = best_svm.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("SVM Accuracy:", accuracy_score(y_test, y_pred))
print()

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Define class labels
class_labels = ['Noise', 'Drone', 'Swarm Drone']

# Print the confusion matrix in count of files
print("Confusion Matrix:")
for i, label in enumerate(class_labels):
    print("{:<12s}".format(label), end="")
    for j in range(len(class_labels)):
        print("{:6d}".format(conf_matrix[i, j]), end=" ")
    print("   True: {}".format(label))

# Display the column labels
print("\nPredicted:  ", end="")
for label in class_labels:
    print("{:<12s}".format(label), end=" ")
print()


Saved model: 1s_melspectrogram_best_svm_model.joblib
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       897
           1       0.99      1.00      1.00       852
           2       0.98      0.98      0.98       868

    accuracy                           0.98      2617
   macro avg       0.98      0.98      0.98      2617
weighted avg       0.98      0.98      0.98      2617

SVM Accuracy: 0.9847153228888039

Confusion Matrix:
Noise          875      5     17    True: Noise
Drone            1    851      0    True: Drone
Swarm Drone     17      0    851    True: Swarm Drone

Predicted:  Noise        Drone        Swarm Drone  


In [ ]:
# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters: ", best_params)
print("Best Score: {:.2f}%".format(best_score*100))

Best Parameters:  {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Best Score: 98.53%


# Mel spectrogram win 1024,  audio = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0] second and run svm based on hyperparameter tuning


In [ ]:
# Define the target durations in seconds
target_durations = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0]

# Define the number of files to take from each folder
num_files = 4361

# Define the window size for Mel spectrogram (in samples)
window_size = 1024


for duration in target_durations:
    features = []
    labels = []

    # Load Mel spectrogram features and assign labels for noise class
    counter = 0
    for filename in os.listdir(noise_folder):
        if filename.endswith('.wav') and counter < num_files:
            file_path = os.path.join(noise_folder, filename)
            audio, sr = librosa.load(file_path, duration=duration)

            mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2)
            mel_spec = librosa.power_to_db(mel_spec)
            mel_spec_features = mel_spec.flatten()
            features.append(mel_spec_features)
            labels.append(0)
            counter += 1

    # Load Mel spectrogram features and assign labels for drone class
    counter = 0
    for filename in os.listdir(drone_folder):
        if filename.endswith('.wav') and counter < num_files:
            file_path = os.path.join(drone_folder, filename)
            audio, sr = librosa.load(file_path, duration=duration)

            mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2)
            mel_spec = librosa.power_to_db(mel_spec)
            mel_spec_features = mel_spec.flatten()
            features.append(mel_spec_features)
            labels.append(1)
            counter += 1

    # Load Mel spectrogram features and assign labels for swarm_drone class
    counter = 0
    for filename in os.listdir(swarm_drone_folder):
        if filename.endswith('.wav') and counter < num_files:
            file_path = os.path.join(swarm_drone_folder, filename)
            audio, sr = librosa.load(file_path, duration=duration)

            mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2)
            mel_spec = librosa.power_to_db(mel_spec)
            mel_spec_features = mel_spec.flatten()
            features.append(mel_spec_features)
            labels.append(2)
            counter += 1

    # Convert features and labels to numpy arrays
    X = np.array(features)
    y = np.array(labels)

    # Perform normalization on the features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train an SVM classifier
    svm = SVC(C=10, kernel='rbf', gamma='scale')
    svm.fit(X_train, y_train)

    # Save the SVM model
    model_name = "{:.1f}s_melspectrogram_svm_model.joblib".format(duration)
    joblib.dump(svm_model, model_name)
    print("Saved model:", model_name)

    # Evaluate the model
    y_pred = svm.predict(X_test)
    print("Duration: {:.1f}s".format(duration))
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("SVM Accuracy:", accuracy_score(y_test, y_pred))
    print()

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Define class labels
    class_labels = ['Noise', 'Drone', 'Swarm Drone']

    # Print the confusion matrix in count of files
    print("Confusion Matrix:")
    for i, label in enumerate(class_labels):
        print("{:<12s}".format(label), end="")
        for j in range(len(class_labels)):
            print("{:6d}".format(conf_matrix[i, j]), end=" ")
        print("   True: {}".format(label))

    # Display the column labels
    print("\nPredicted:  ", end="")
    for label in class_labels:
        print("{:<12s}".format(label), end=" ")
    print()
    # Add line markers
    print('\n' + '-'*40 + '\n')


Saved model: 0.1s_melspectrogram_svm_model.joblib
Duration: 0.1s
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.95      0.96       897
           1       0.96      0.99      0.98       852
           2       0.97      0.97      0.97       868

    accuracy                           0.97      2617
   macro avg       0.97      0.97      0.97      2617
weighted avg       0.97      0.97      0.97      2617

SVM Accuracy: 0.9709591134887275

Confusion Matrix:
Noise          851     25     21    True: Noise
Drone            4    846      2    True: Drone
Swarm Drone     18      6    844    True: Swarm Drone

Predicted:  Noise        Drone        Swarm Drone  

----------------------------------------

Saved model: 0.2s_melspectrogram_svm_model.joblib
Duration: 0.2s
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.97      0.97       897
           1       0.98      0.99   

# essemble learning 1's audio

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
import os
import librosa
import numpy as np
import joblib
from scipy import stats
# Define the number of files to take from each folder
num_files = 4361
# Define the window size for Mel spectrogram (in samples)
window_size = 1024

# Set the paths to your audio data folders
drone_folder = '/content/drive/MyDrive/new_training_dataset/drone'
swarm_drone_folder = '/content/drive/MyDrive/new_training_dataset/Copy_of_swarm'
noise_folder = '/content/drive/MyDrive/new_training_dataset/noise_full'

features = []
labels = []
# Load Mel spectrogram features and assign labels for noise class
counter = 0
for filename in os.listdir(noise_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(noise_folder, filename)
        audio, sr = librosa.load(file_path, duration=1.0)
        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2)
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec_features = mel_spec.flatten()
        features.append(mel_spec_features)
        labels.append(0)
        counter += 1

# Load Mel spectrogram features and assign labels for drone class
counter = 0
for filename in os.listdir(drone_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(drone_folder, filename)
        audio, sr = librosa.load(file_path, duration=1.0)

        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2)
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec_features = mel_spec.flatten()
        features.append(mel_spec_features)
        labels.append(1)
        counter += 1

# Load Mel spectrogram features and assign labels for swarm_drone class
counter = 0
for filename in os.listdir(swarm_drone_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(swarm_drone_folder, filename)
        audio, sr = librosa.load(file_path, duration=1.0)

        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2)
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec_features = mel_spec.flatten()
        features.append(mel_spec_features)
        labels.append(2)
        counter += 1

# Convert features and labels to numpy arrays
X = np.array(features)
y = np.array(labels)

# Apply Z-score normalization to the features
X = stats.zscore(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
# import joblib

# # Save the training and testing data
# joblib.dump(X_train, '/content/drive/MyDrive/X_train.pkl')
# joblib.dump(X_test, '/content/drive/MyDrive/X_test.pkl')
# joblib.dump(y_train, '/content/drive/MyDrive/y_train.pkl')
# joblib.dump(y_test, '/content/drive/MyDrive/y_test.pkl')


In [ ]:
# Load the training and testing data
X_train = joblib.load('/content/drive/MyDrive/X_train.pkl')
X_test = joblib.load('/content/drive/MyDrive/X_test.pkl')
y_train = joblib.load('/content/drive/MyDrive/y_train.pkl')
y_test = joblib.load('/content/drive/MyDrive/y_test.pkl')


In [ ]:
X_train

array([[ 0.8016668 ,  1.0410682 ,  0.40220338, ...,  0.5198161 ,
         0.5640177 ,  0.42611715],
       [ 0.271937  ,  0.47145882,  0.29749885, ...,  0.6248751 ,
         0.38344735, -0.19633175],
       [-0.7420022 , -2.0134604 , -2.1483076 , ...,  0.95035696,
         0.8550978 ,  1.1064308 ],
       ...,
       [-0.21524158,  0.05745608,  0.13602677, ...,  0.5917589 ,
         0.8997477 ,  0.4210272 ],
       [-0.14884827, -2.158494  , -2.1679368 , ..., -0.585845  ,
        -0.5854221 ,  1.0239265 ],
       [ 0.09829998,  1.0827498 ,  1.0292366 , ...,  0.39389902,
         0.9702755 , -0.12272917]], dtype=float32)

In [ ]:
# Load the saved SVM model
svm_model_path = "/content/drive/MyDrive/1s_melspectrogram_best_svm_model.joblib"
best_svm = joblib.load(svm_model_path)
# Define base classifiers
random_forest = RandomForestClassifier(n_estimators=100)
gradient_boosting = GradientBoostingClassifier()

# Create list of base classifiers including the saved SVM model
classifiers = [('SVM', best_svm), ('Random Forest', random_forest), ('Gradient Boosting', gradient_boosting)]

# Create ensemble classifier
ensemble_classifier = VotingClassifier(estimators=classifiers, voting='hard')
# Train the ensemble classifier
ensemble_classifier.fit(X_train, y_train)
# Evaluate the ensemble classifier
y_pred_ensemble = ensemble_classifier.predict(X_test)
print("Ensemble Classifier Accuracy:", accuracy_score(y_test, y_pred_ensemble))
print("Classification Report:")
print(classification_report(y_test, y_pred_ensemble))

# Save the ensemble classifier
ensemble_model_name = "one_sec_ensemble_model.joblib"
joblib.dump(ensemble_classifier, ensemble_model_name)
print("Saved ensemble model:", ensemble_model_name)

Ensemble Classifier Accuracy: 0.9801299197554452
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       897
           1       0.98      1.00      0.99       852
           2       0.98      0.98      0.98       868

    accuracy                           0.98      2617
   macro avg       0.98      0.98      0.98      2617
weighted avg       0.98      0.98      0.98      2617

Saved ensemble model: ensemble_model.joblib


In [ ]:
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
# Define the random forest classifier
random_forest = RandomForestClassifier(n_estimators=100)

# Train the random forest classifier
random_forest.fit(X_train, y_train)

# Predict using the random forest classifier
y_pred_random_forest = random_forest.predict(X_test)

# Calculate accuracy for random forest
accuracy_rf = accuracy_score(y_test, y_pred_random_forest)
print("Random Forest Accuracy:", accuracy_rf)
print("Classification Report (Random Forest):")
print(classification_report(y_test, y_pred_random_forest))

# Save the random forest classifier
rf_model_path = "/content/drive/MyDrive/1s_random_forest_model.joblib"
joblib.dump(random_forest, rf_model_path)
print("Random Forest model saved:", rf_model_path)

Random Forest Accuracy: 0.9763087504776462
Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       897
           1       0.97      1.00      0.98       852
           2       0.98      0.98      0.98       868

    accuracy                           0.98      2617
   macro avg       0.98      0.98      0.98      2617
weighted avg       0.98      0.98      0.98      2617

Random Forest model saved: /content/drive/MyDrive/1s_random_forest_model.joblib


In [ ]:
# Define the gradient boosting classifier
gradient_boosting = GradientBoostingClassifier()

# Train the gradient boosting classifier
gradient_boosting.fit(X_train, y_train)

# Predict using the gradient boosting classifier
y_pred_gradient_boosting = gradient_boosting.predict(X_test)

# Calculate accuracy for gradient boosting
accuracy_gb = accuracy_score(y_test, y_pred_gradient_boosting)
print("Gradient Boosting Accuracy:", accuracy_gb)
print("Classification Report (Gradient Boosting):")
print(classification_report(y_test, y_pred_gradient_boosting))

# Save the gradient boosting classifier
gb_model_path = "/content/drive/MyDrive/1s_gradient_boosting_model.joblib"
joblib.dump(gradient_boosting, gb_model_path)
print("Gradient Boosting model saved:", gb_model_path)

Gradient Boosting Accuracy: 0.9743981658387466
Classification Report (Gradient Boosting):
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       897
           1       0.97      1.00      0.98       852
           2       0.98      0.97      0.98       868

    accuracy                           0.97      2617
   macro avg       0.97      0.97      0.97      2617
weighted avg       0.97      0.97      0.97      2617

Gradient Boosting model saved: /content/drive/MyDrive/1s_gradient_boosting_model.joblib


In [ ]:
# Load the random forest model
random_forest = joblib.load("/content/drive/MyDrive/1s_random_forest_model.joblib")

# Load the gradient boosting model
gradient_boosting = joblib.load("/content/drive/MyDrive/1s_gradient_boosting_model.joblib")
# Load the saved SVM model
svm_model_path = "/content/drive/MyDrive/1s_melspectrogram_best_svm_model.joblib"
best_svm = joblib.load(svm_model_path)

# Create list of base classifiers including the saved SVM model
classifiers = [('SVM', best_svm), ('Random Forest', random_forest), ('Gradient Boosting', gradient_boosting)]

In [ ]:
# Predict using the loaded random forest model
y_pred_random_forest = random_forest.predict(X_test)
# Generate the confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred_random_forest)
print("Confusion Matrix (Random Forest):")
print(confusion_mat)

Confusion Matrix (Random Forest):
[[858  26  13]
 [  2 850   0]
 [ 19   2 847]]


In [ ]:
# Predict using the loaded Gradient Boosting model
y_pred_gradient_boosting = gradient_boosting.predict(X_test)
# Generate the confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred_gradient_boosting)
print("Confusion Matrix (Gradient Boosting):")
print(confusion_mat)

Confusion Matrix (Gradient Boosting):
[[859  22  16]
 [  3 849   0]
 [ 18   8 842]]


In [ ]:
# from sklearn.externals import joblib

# Create ensemble classifier (Voting)
ensemble_voting = VotingClassifier(estimators=classifiers, voting='hard')

# Train the ensemble classifier (Voting)
ensemble_voting.fit(X_train, y_train)

# Evaluate the ensemble classifier (Voting)
y_pred_voting = ensemble_voting.predict(X_test)
print("Voting Classifier Accuracy:", accuracy_score(y_test, y_pred_voting))
print("Classification Report (Voting):")
print(classification_report(y_test, y_pred_voting))

# Save the ensemble voting classifier model
joblib.dump(ensemble_voting, 'ensemble_voting_model.pkl')

Voting Classifier Accuracy: 0.9801299197554452
Classification Report (Voting):
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       897
           1       0.98      1.00      0.99       852
           2       0.98      0.98      0.98       868

    accuracy                           0.98      2617
   macro avg       0.98      0.98      0.98      2617
weighted avg       0.98      0.98      0.98      2617



['ensemble_voting_model.pkl']

In [ ]:
# # Create Bagging ensemble with SVM as the base estimator
# ensemble_bagging_svm = BaggingClassifier(base_estimator=best_svm, n_estimators=10)
# ensemble_bagging_svm.fit(X_train, y_train)
# y_pred_bagging_svm = ensemble_bagging_svm.predict(X_test)

# # Create Bagging ensemble with Random Forest as the base estimator
# ensemble_bagging_rf = BaggingClassifier(base_estimator=random_forest, n_estimators=10)
# ensemble_bagging_rf.fit(X_train, y_train)
# y_pred_bagging_rf = ensemble_bagging_rf.predict(X_test)

# # Create Bagging ensemble with Gradient Boosting as the base estimator
# ensemble_bagging_gb = BaggingClassifier(base_estimator=gradient_boosting, n_estimators=10)
# ensemble_bagging_gb.fit(X_train, y_train)
# y_pred_bagging_gb = ensemble_bagging_gb.predict(X_test)

# # Evaluate Bagging ensembles,
# print("Bagging with SVM Classifier Accuracy:", accuracy_score(y_test, y_pred_bagging_svm))
# print("Classification Report (Bagging with SVM):")
# print(classification_report(y_test, y_pred_bagging_svm))

# print("Bagging with Random Forest Classifier Accuracy:", accuracy_score(y_test, y_pred_bagging_rf))
# print("Classification Report (Bagging with Random Forest):")
# print(classification_report(y_test, y_pred_bagging_rf))

# print("Bagging with Gradient Boosting Classifier Accuracy:", accuracy_score(y_test, y_pred_bagging_gb))
# print("Classification Report (Bagging with Gradient Boosting):")
# print(classification_report(y_test, y_pred_bagging_gb))

# # Save Bagging ensemble with SVM model
# joblib.dump(ensemble_bagging_svm, 'bagging_svm_model.pkl')

# # Save Bagging ensemble with Random Forest model
# joblib.dump(ensemble_bagging_rf, 'bagging_rf_model.pkl')

# # Save Bagging ensemble with Gradient Boosting model
# joblib.dump(ensemble_bagging_gb, 'bagging_gb_model.pkl')


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [ ]:
#1.0 sec
from sklearn.ensemble import BaggingClassifier

# Create Bagging ensemble with SVM as the base estimator
ensemble_bagging_svm = BaggingClassifier(base_estimator=best_svm, n_estimators=10)
ensemble_bagging_svm.fit(X_train, y_train)
y_pred_bagging_svm = ensemble_bagging_svm.predict(X_test)

print("Bagging with SVM Classifier Accuracy:", accuracy_score(y_test, y_pred_bagging_svm))
print("Classification Report (Bagging with SVM):")
print(classification_report(y_test, y_pred_bagging_svm))
joblib.dump(ensemble_bagging_svm, '1.0bagging_svm_model.pkl')


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Bagging with SVM Classifier Accuracy: 0.9866259075277035
Classification Report (Bagging with SVM):
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       897
           1       0.99      1.00      0.99       852
           2       0.99      0.98      0.98       868

    accuracy                           0.99      2617
   macro avg       0.99      0.99      0.99      2617
weighted avg       0.99      0.99      0.99      2617



['1.0bagging_svm_model.pkl']

In [ ]:
# on 0.6 second audio file

from sklearn.ensemble import BaggingClassifier

# Create Bagging ensemble with SVM as the base estimator
ensemble_bagging_svm = BaggingClassifier(base_estimator=best_svm, n_estimators=10)
ensemble_bagging_svm.fit(X_train, y_train)
y_pred_bagging_svm = ensemble_bagging_svm.predict(X_test)

# Create Bagging ensemble with Random Forest as the base estimator
ensemble_bagging_rf = BaggingClassifier(base_estimator=random_forest, n_estimators=10)
ensemble_bagging_rf.fit(X_train, y_train)
y_pred_bagging_rf = ensemble_bagging_rf.predict(X_test)

# Create Bagging ensemble with Gradient Boosting as the base estimator
ensemble_bagging_gb = BaggingClassifier(base_estimator=gradient_boosting, n_estimators=10)
ensemble_bagging_gb.fit(X_train, y_train)
y_pred_bagging_gb = ensemble_bagging_gb.predict(X_test)

# Evaluate Bagging ensembles,
print("Bagging with SVM Classifier Accuracy:", accuracy_score(y_test, y_pred_bagging_svm))
print("Classification Report (Bagging with SVM):")
print(classification_report(y_test, y_pred_bagging_svm))

print("Bagging with Random Forest Classifier Accuracy:", accuracy_score(y_test, y_pred_bagging_rf))
print("Classification Report (Bagging with Random Forest):")
print(classification_report(y_test, y_pred_bagging_rf))

print("Bagging with Gradient Boosting Classifier Accuracy:", accuracy_score(y_test, y_pred_bagging_gb))
print("Classification Report (Bagging with Gradient Boosting):")
print(classification_report(y_test, y_pred_bagging_gb))

# Save Bagging ensemble with SVM model
joblib.dump(ensemble_bagging_svm, '0.6bagging_svm_model.pkl')

# Save Bagging ensemble with Random Forest model
joblib.dump(ensemble_bagging_rf, '0.6bagging_rf_model.pkl')

# Save Bagging ensemble with Gradient Boosting model
joblib.dump(ensemble_bagging_gb, '0.6bagging_gb_model.pkl')


Bagging with SVM Classifier Accuracy: 0.9778372181887658
Classification Report (Bagging with SVM):
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       897
           1       1.00      0.98      0.99       852
           2       0.98      0.97      0.98       868

    accuracy                           0.98      2617
   macro avg       0.98      0.98      0.98      2617
weighted avg       0.98      0.98      0.98      2617

Bagging with Random Forest Classifier Accuracy: 0.9690485288498281
Classification Report (Bagging with Random Forest):
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       897
           1       0.98      0.98      0.98       852
           2       0.97      0.97      0.97       868

    accuracy                           0.97      2617
   macro avg       0.97      0.97      0.97      2617
weighted avg       0.97      0.97      0.97      2617

Bagging with Gradien

['0.6bagging_gb_model.pkl']

In [ ]:
#bagging

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, classification_report

# Create a linear SVM classifier
linear_svm = SVC(C = 1, kernel='linear')

# Create Bagging ensemble with linear SVM as the base estimator
ensemble_bagging_linear_svm = BaggingClassifier(base_estimator=linear_svm, n_estimators=10)
ensemble_bagging_linear_svm.fit(X_train, y_train)
y_pred_bagging_linear_svm = ensemble_bagging_linear_svm.predict(X_test)

# Evaluate Bagging ensemble with linear SVM
print("Bagging with Linear SVM Classifier Accuracy:", accuracy_score(y_test, y_pred_bagging_linear_svm))
print("Classification Report (Bagging with Linear SVM):")
print(classification_report(y_test, y_pred_bagging_linear_svm))


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Bagging with Linear SVM Classifier Accuracy: 0.9564386702330914
Classification Report (Bagging with Linear SVM):
              precision    recall  f1-score   support

           0       0.93      0.94      0.94       897
           1       0.96      0.96      0.96       852
           2       0.98      0.96      0.97       868

    accuracy                           0.96      2617
   macro avg       0.96      0.96      0.96      2617
weighted avg       0.96      0.96      0.96      2617



In [ ]:
# Create a cubic SVM classifier
cubic_svm = SVC(C=1, kernel='poly', degree=3)

# Create Bagging ensemble with cubic SVM as the base estimator
ensemble_bagging_cubic_svm = BaggingClassifier(base_estimator=cubic_svm, n_estimators=10)
ensemble_bagging_cubic_svm.fit(X_train, y_train)
y_pred_bagging_cubic_svm = ensemble_bagging_cubic_svm.predict(X_test)

# Evaluate Bagging ensemble with cubic SVM
print("Bagging with Cubic SVM Classifier Accuracy:", accuracy_score(y_test, y_pred_bagging_cubic_svm))
print("Classification Report (Bagging with Cubic SVM):")
print(classification_report(y_test, y_pred_bagging_cubic_svm))


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Bagging with Cubic SVM Classifier Accuracy: 0.9419182269774551
Classification Report (Bagging with Cubic SVM):
              precision    recall  f1-score   support

           0       0.97      0.90      0.94       897
           1       0.88      1.00      0.93       852
           2       0.99      0.93      0.96       868

    accuracy                           0.94      2617
   macro avg       0.95      0.94      0.94      2617
weighted avg       0.95      0.94      0.94      2617



In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, classification_report

# Create a linear SVM classifier
linear_svm = SVC(C = 10, kernel='linear')

# Create Bagging ensemble with linear SVM as the base estimator
ensemble_bagging_linear_svm = BaggingClassifier(base_estimator=linear_svm, n_estimators=10)
ensemble_bagging_linear_svm.fit(X_train, y_train)
y_pred_bagging_linear_svm = ensemble_bagging_linear_svm.predict(X_test)

# Evaluate Bagging ensemble with linear SVM
print("Bagging with Linear SVM Classifier Accuracy:", accuracy_score(y_test, y_pred_bagging_linear_svm))
print("Classification Report (Bagging with Linear SVM):")
print(classification_report(y_test, y_pred_bagging_linear_svm))


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Bagging with Linear SVM Classifier Accuracy: 0.9564386702330914
Classification Report (Bagging with Linear SVM):
              precision    recall  f1-score   support

           0       0.93      0.95      0.94       897
           1       0.97      0.96      0.96       852
           2       0.98      0.96      0.97       868

    accuracy                           0.96      2617
   macro avg       0.96      0.96      0.96      2617
weighted avg       0.96      0.96      0.96      2617



In [ ]:
# Create a cubic SVM classifier
cubic_svm = SVC(C=10, kernel='poly', degree=3)

# Create Bagging ensemble with cubic SVM as the base estimator
ensemble_bagging_cubic_svm = BaggingClassifier(base_estimator=cubic_svm, n_estimators=10)
ensemble_bagging_cubic_svm.fit(X_train, y_train)
y_pred_bagging_cubic_svm = ensemble_bagging_cubic_svm.predict(X_test)

# Evaluate Bagging ensemble with cubic SVM
print("Bagging with Cubic SVM Classifier Accuracy:", accuracy_score(y_test, y_pred_bagging_cubic_svm))
print("Classification Report (Bagging with Cubic SVM):")
print(classification_report(y_test, y_pred_bagging_cubic_svm))


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Bagging with Cubic SVM Classifier Accuracy: 0.9621704241497898
Classification Report (Bagging with Cubic SVM):
              precision    recall  f1-score   support

           0       0.98      0.92      0.95       897
           1       0.92      1.00      0.96       852
           2       0.99      0.97      0.98       868

    accuracy                           0.96      2617
   macro avg       0.96      0.96      0.96      2617
weighted avg       0.96      0.96      0.96      2617



In [ ]:
#mean along coloumn i.e axis time axis

# mfcc mean along coloumn i.e axis time axis

In [ ]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from joblib import dump

# # Define the window size for MFCC (in samples)
# window_size = 1024
# # Define the number of files to consider from each folder
# num_files = 4361

# # Define the paths to the data folders
# noise_folder = 'path_to_noise_folder'
# drone_folder = 'path_to_drone_folder'
# swarm_drone_folder = 'path_to_swarm_drone_folder'

# Initialize empty lists for features and labels
features = []
labels = []

# Load MFCC features and assign labels for noise class
counter = 0
for filename in os.listdir(noise_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(noise_folder, filename)
        audio, sr = librosa.load(file_path)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2)
        mfcc_mean = np.mean(mfcc, axis=1)  # Calculate the mean along the columns
        features.append(mfcc_mean)
        labels.append(0)
        counter += 1

# Load MFCC features and assign labels for drone class
counter = 0
for filename in os.listdir(drone_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(drone_folder, filename)
        audio, sr = librosa.load(file_path)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2)
        mfcc_mean = np.mean(mfcc, axis=1)  # Calculate the mean along the columns
        features.append(mfcc_mean)
        labels.append(1)
        counter += 1

# Load MFCC features and assign labels for swarm_drone class
counter = 0
for filename in os.listdir(swarm_drone_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(swarm_drone_folder, filename)
        audio, sr = librosa.load(file_path)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2)
        mfcc_mean = np.mean(mfcc, axis=1)  # Calculate the mean along the columns
        features.append(mfcc_mean)
        labels.append(2)
        counter += 1

# Convert features and labels to numpy arrays
X = np.array(features)
y = np.array(labels)

# Perform normalization on the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the SVM model with the best hyperparameters
svm_model = SVC(C=10, degree=2, gamma='scale', kernel='rbf')

# Train the model on the training data
svm_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = svm_model.predict(X_test)

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Define class labels
class_labels = ['Noise', 'Drone', 'Swarm Drone']

# Print the confusion matrix in count of files
print("Confusion Matrix:")
for i, label in enumerate(class_labels):
    print("{:<12s}".format(label), end="")
    for j in range(len(class_labels)):
        print("{:6d}".format(conf_matrix[i, j]), end=" ")
    print("   True: {}".format(label))

# Display the column labels
print("\nPredicted:  ", end="")
for label in class_labels:
    print("{:<12s}".format(label), end=" ")
print()

# Save the trained model
model_filename = "mfcc_mean_1s.joblib"
dump(svm_model, model_filename)
print("Model saved as", model_filename)


Accuracy: 98.47%
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       897
           1       0.99      0.99      0.99       852
           2       1.00      0.97      0.98       868

    accuracy                           0.98      2617
   macro avg       0.99      0.98      0.98      2617
weighted avg       0.99      0.98      0.98      2617

Confusion Matrix:
Noise          891      6      0    True: Noise
Drone            7    845      0    True: Drone
Swarm Drone     27      0    841    True: Swarm Drone

Predicted:  Noise        Drone        Swarm Drone  
Model saved as mfcc_mean_1s.joblib


# Mel_spec using mean along coloumn i.e axis time axis

In [ ]:
# n_mel = 128(default)
features = []
labels = []

# Load Mel spectrogram features and assign labels for noise class
counter = 0
for filename in os.listdir(noise_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(noise_folder, filename)
        audio, sr = librosa.load(file_path, duration=1.0)

        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2)
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec_mean = np.mean(mel_spec, axis=1)  # Calculate the mean along the columns
        features.append(mel_spec_mean)
        labels.append(0)
        counter += 1

# Load Mel spectrogram features and assign labels for drone class
counter = 0
for filename in os.listdir(drone_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(drone_folder, filename)
        audio, sr = librosa.load(file_path, duration=1.0)

        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2)
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec_mean = np.mean(mel_spec, axis=1)  # Calculate the mean along the columns
        features.append(mel_spec_mean)
        labels.append(1)
        counter += 1

# Load Mel spectrogram features and assign labels for swarm_drone class
counter = 0
for filename in os.listdir(swarm_drone_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(swarm_drone_folder, filename)
        audio, sr = librosa.load(file_path, duration=1.0)

        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2)
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec_mean = np.mean(mel_spec, axis=1)  # Calculate the mean along the columns
        features.append(mel_spec_mean)
        labels.append(2)
        counter += 1

# Convert features and labels to numpy arrays
X = np.array(features)
y = np.array(labels)

# Perform normalization on the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the SVM classifier with the best parameters
svm = SVC(C=10, degree=2, gamma='scale', kernel='rbf')

# Train the model
svm.fit(X_train, y_train)

# Save the trained model
model_name = "1s_melspectrogram_mean_best_svm_model.joblib"
joblib.dump(svm, model_name)
print("Saved model:", model_name)

# Evaluate the model
y_pred = svm.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("SVM Accuracy:", accuracy_score(y_test, y_pred))
print()

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Define class labels
class_labels = ['Noise', 'Drone', 'Swarm Drone']

# Print the confusion matrix in count of files
print("Confusion Matrix:")
for i, label in enumerate(class_labels):
    print("{:<12s}".format(label), end="")
    for j in range(len(class_labels)):
        print("{:6d}".format(conf_matrix[i, j]), end=" ")
    print("   True: {}".format(label))

# Display the column labels
print("\nPredicted:  ", end="")
for label in class_labels:
    print("{:<12s}".format(label), end=" ")
print()


Saved model: 1s_melspectrogram_mean_best_svm_model.joblib
Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       897
           1       1.00      0.99      0.99       852
           2       1.00      0.97      0.98       868

    accuracy                           0.99      2617
   macro avg       0.99      0.99      0.99      2617
weighted avg       0.99      0.99      0.99      2617

SVM Accuracy: 0.9854795567443638

Confusion Matrix:
Noise          893      2      2    True: Noise
Drone            8    844      0    True: Drone
Swarm Drone     26      0    842    True: Swarm Drone

Predicted:  Noise        Drone        Swarm Drone  


In [ ]:
n_mels=256

features = []
labels = []

# Load Mel spectrogram features and assign labels for noise class
counter = 0
for filename in os.listdir(noise_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(noise_folder, filename)
        audio, sr = librosa.load(file_path, duration=1.0)

        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2, n_mels=n_mels)
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec_mean = np.mean(mel_spec, axis=1)  # Calculate the mean along the columns
        features.append(mel_spec_mean)
        labels.append(0)
        counter += 1

# Load Mel spectrogram features and assign labels for drone class
counter = 0
for filename in os.listdir(drone_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(drone_folder, filename)
        audio, sr = librosa.load(file_path, duration=1.0)

        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2, n_mels=n_mels)
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec_mean = np.mean(mel_spec, axis=1)  # Calculate the mean along the columns
        features.append(mel_spec_mean)
        labels.append(1)
        counter += 1

# Load Mel spectrogram features and assign labels for swarm_drone class
counter = 0
for filename in os.listdir(swarm_drone_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(swarm_drone_folder, filename)
        audio, sr = librosa.load(file_path, duration=1.0)

        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2, n_mels=n_mels)
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec_mean = np.mean(mel_spec, axis=1)  # Calculate the mean along the columns
        features.append(mel_spec_mean)
        labels.append(2)
        counter += 1

# Convert features and labels to numpy arrays
X = np.array(features)
y = np.array(labels)

# Perform normalization on the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the SVM classifier with the best parameters
svm = SVC(C=10, degree=2, gamma='scale', kernel='rbf')

# Train the model
svm.fit(X_train, y_train)

# Save the trained model
model_name = "1s_melspectrogram_mean_best_svm_model.joblib"
joblib.dump(svm, model_name)
print("Saved model:", model_name)

# Evaluate the model
y_pred = svm.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("SVM Accuracy:", accuracy_score(y_test, y_pred))
print()

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Define class labels
class_labels = ['Noise', 'Drone', 'Swarm Drone']

# Print the confusion matrix in count of files
print("Confusion Matrix:")
for i, label in enumerate(class_labels):
    print("{:<12s}".format(label), end="")
    for j in range(len(class_labels)):
        print("{:6d}".format(conf_matrix[i, j]), end=" ")
    print("   True: {}".format(label))

# Display the column labels
print("\nPredicted:  ", end="")
for label in class_labels:
    print("{:<12s}".format(label), end=" ")
print()


Saved model: 1s_melspectrogram_mean_best_svm_model.joblib
Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       897
           1       1.00      0.99      1.00       852
           2       1.00      0.97      0.98       868

    accuracy                           0.99      2617
   macro avg       0.99      0.99      0.99      2617
weighted avg       0.99      0.99      0.99      2617

SVM Accuracy: 0.9870080244554834

Confusion Matrix:
Noise          894      2      1    True: Noise
Drone            5    847      0    True: Drone
Swarm Drone     26      0    842    True: Swarm Drone

Predicted:  Noise        Drone        Swarm Drone  


In [ ]:
n_mels=64

features = []
labels = []

# Load Mel spectrogram features and assign labels for noise class
counter = 0
for filename in os.listdir(noise_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(noise_folder, filename)
        audio, sr = librosa.load(file_path, duration=1.0)

        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2, n_mels=n_mels)
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec_mean = np.mean(mel_spec, axis=1)  # Calculate the mean along the columns
        features.append(mel_spec_mean)
        labels.append(0)
        counter += 1

# Load Mel spectrogram features and assign labels for drone class
counter = 0
for filename in os.listdir(drone_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(drone_folder, filename)
        audio, sr = librosa.load(file_path, duration=1.0)

        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2, n_mels=n_mels)
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec_mean = np.mean(mel_spec, axis=1)  # Calculate the mean along the columns
        features.append(mel_spec_mean)
        labels.append(1)
        counter += 1

# Load Mel spectrogram features and assign labels for swarm_drone class
counter = 0
for filename in os.listdir(swarm_drone_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(swarm_drone_folder, filename)
        audio, sr = librosa.load(file_path, duration=1.0)

        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2, n_mels=n_mels)
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec_mean = np.mean(mel_spec, axis=1)  # Calculate the mean along the columns
        features.append(mel_spec_mean)
        labels.append(2)
        counter += 1

# Convert features and labels to numpy arrays
X = np.array(features)
y = np.array(labels)

# Perform normalization on the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the SVM classifier with the best parameters
svm = SVC(C=10, degree=2, gamma='scale', kernel='rbf')

# Train the model
svm.fit(X_train, y_train)

# Save the trained model
model_name = "1s_melspectrogram_mean_best_svm_model_64.joblib"
joblib.dump(svm, model_name)
print("Saved model:", model_name)

# Evaluate the model
y_pred = svm.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("SVM Accuracy:", accuracy_score(y_test, y_pred))
print()

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Define class labels
class_labels = ['Noise', 'Drone', 'Swarm Drone']

# Print the confusion matrix in count of files
print("Confusion Matrix:")
for i, label in enumerate(class_labels):
    print("{:<12s}".format(label), end="")
    for j in range(len(class_labels)):
        print("{:6d}".format(conf_matrix[i, j]), end=" ")
    print("   True: {}".format(label))

# Display the column labels
print("\nPredicted:  ", end="")
for label in class_labels:
    print("{:<12s}".format(label), end=" ")
print()


Saved model: 1s_melspectrogram_mean_best_svm_model_64.joblib
Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       897
           1       1.00      0.99      0.99       852
           2       1.00      0.97      0.98       868

    accuracy                           0.98      2617
   macro avg       0.99      0.98      0.98      2617
weighted avg       0.99      0.98      0.98      2617

SVM Accuracy: 0.9847153228888039

Confusion Matrix:
Noise          893      2      2    True: Noise
Drone            9    843      0    True: Drone
Swarm Drone     27      0    841    True: Swarm Drone

Predicted:  Noise        Drone        Swarm Drone  


In [ ]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from joblib import dump

# Define the window size for MFCC (in samples)
window_size = 1024

# Define the number of files to consider from each folder
num_files = 4361

# Set the paths to your audio data folders
drone_folder = '/content/drive/MyDrive/new_training_dataset/drone'
swarm_drone_folder = '/content/drive/MyDrive/new_training_dataset/Copy_of_swarm'
noise_folder = '/content/drive/MyDrive/new_training_dataset/noise_44100'

# Initialize empty lists for features and labels
features = []
labels = []

n_mels=512


# Load Mel spectrogram features and assign labels for noise class
counter = 0
for filename in os.listdir(noise_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(noise_folder, filename)
        audio, sr = librosa.load(file_path, duration=1.0)

        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2, n_mels=n_mels)
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec_mean = np.mean(mel_spec, axis=1)  # Calculate the mean along the columns
        features.append(mel_spec_mean)
        labels.append(0)
        counter += 1

# Load Mel spectrogram features and assign labels for drone class
counter = 0
for filename in os.listdir(drone_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(drone_folder, filename)
        audio, sr = librosa.load(file_path, duration=1.0)

        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2, n_mels=n_mels)
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec_mean = np.mean(mel_spec, axis=1)  # Calculate the mean along the columns
        features.append(mel_spec_mean)
        labels.append(1)
        counter += 1

# Load Mel spectrogram features and assign labels for swarm_drone class
counter = 0
for filename in os.listdir(swarm_drone_folder):
    if filename.endswith('.wav') and counter < num_files:
        file_path = os.path.join(swarm_drone_folder, filename)
        audio, sr = librosa.load(file_path, duration=1.0)

        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size, hop_length=window_size // 2, n_mels=n_mels)
        mel_spec = librosa.power_to_db(mel_spec)
        mel_spec_mean = np.mean(mel_spec, axis=1)  # Calculate the mean along the columns
        features.append(mel_spec_mean)
        labels.append(2)
        counter += 1

# Convert features and labels to numpy arrays
X = np.array(features)
y = np.array(labels)

# Perform normalization on the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the SVM classifier with the best parameters
svm = SVC(C=10, degree=2, gamma='scale', kernel='rbf')

# Train the model
svm.fit(X_train, y_train)


# Evaluate the model
y_pred = svm.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("SVM Accuracy:", accuracy_score(y_test, y_pred))
print()

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Define class labels
class_labels = ['Noise', 'Drone', 'Swarm Drone']

# Print the confusion matrix in count of files
print("Confusion Matrix:")
for i, label in enumerate(class_labels):
    print("{:<12s}".format(label), end="")
    for j in range(len(class_labels)):
        print("{:6d}".format(conf_matrix[i, j]), end=" ")
    print("   True: {}".format(label))

# Display the column labels
print("\nPredicted:  ", end="")
for label in class_labels:
    print("{:<12s}".format(label), end=" ")
print()


Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       897
           1       1.00      1.00      1.00       852
           2       1.00      0.97      0.98       868

    accuracy                           0.99      2617
   macro avg       0.99      0.99      0.99      2617
weighted avg       0.99      0.99      0.99      2617

SVM Accuracy: 0.9873901413832633

Confusion Matrix:
Noise          897      0      0    True: Noise
Drone            3    849      0    True: Drone
Swarm Drone     30      0    838    True: Swarm Drone

Predicted:  Noise        Drone        Swarm Drone  


In [ ]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Define the window size for MFCC (in samples)
window_size = 1024

# Define the number of files to consider from each folder
num_files = 4361

# Set the paths to your audio data folders
drone_folder = '/content/drive/MyDrive/new_training_dataset/drone'
swarm_drone_folder = '/content/drive/MyDrive/new_training_dataset/Copy_of_swarm'
noise_folder = '/content/drive/MyDrive/new_training_dataset/noise_44100'

# Initialize empty lists for features and labels
features = []
labels = []

n_mels = 128

# Define durations to be evaluated
durations = [0.1, 0.2, 0.4, 0.6, 0.8]

for duration in durations:
    print(f"Evaluating duration: {duration} seconds")

    # Load Mel spectrogram features and assign labels for noise class
    counter = 0
    for filename in os.listdir(noise_folder):
        if filename.endswith('.wav') and counter < num_files:
            file_path = os.path.join(noise_folder, filename)
            audio, sr = librosa.load(file_path, duration=duration)

            mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size,
                                                      hop_length=window_size // 2, n_mels=n_mels)
            mel_spec = librosa.power_to_db(mel_spec)
            mel_spec_mean = np.mean(mel_spec, axis=1)  # Calculate the mean along the columns
            features.append(mel_spec_mean)
            labels.append(0)
            counter += 1

    # Load Mel spectrogram features and assign labels for drone class
    counter = 0
    for filename in os.listdir(drone_folder):
        if filename.endswith('.wav') and counter < num_files:
            file_path = os.path.join(drone_folder, filename)
            audio, sr = librosa.load(file_path, duration=duration)

            mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size,
                                                      hop_length=window_size // 2, n_mels=n_mels)
            mel_spec = librosa.power_to_db(mel_spec)
            mel_spec_mean = np.mean(mel_spec, axis=1)  # Calculate the mean along the columns
            features.append(mel_spec_mean)
            labels.append(1)
            counter += 1

    # Load Mel spectrogram features and assign labels for swarm_drone class
    counter = 0
    for filename in os.listdir(swarm_drone_folder):
        if filename.endswith('.wav') and counter < num_files:
            file_path = os.path.join(swarm_drone_folder, filename)
            audio, sr = librosa.load(file_path, duration=duration)

            mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=window_size,
                                                      hop_length=window_size // 2, n_mels=n_mels)
            mel_spec = librosa.power_to_db(mel_spec)
            mel_spec_mean = np.mean(mel_spec, axis=1)  # Calculate the mean along the columns
            features.append(mel_spec_mean)
            labels.append(2)
            counter += 1

    # Convert features and labels to numpy arrays
    X = np.array(features)
    y = np.array(labels)

    # Perform normalization on the features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the SVM classifier with the best parameters
    svm = SVC(C=10, degree=2, gamma='scale', kernel='rbf')

    # Train the model
    svm.fit(X_train, y_train)

    # Evaluate the model
    y_pred = svm.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("SVM Accuracy:", accuracy)
    print()

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Define class labels
    class_labels = ['Noise', 'Drone', 'Swarm Drone']

    # Print the confusion matrix in count of files
    print("Confusion Matrix:")
    for i, label in enumerate(class_labels):
        print("{:<12s}".format(label), end="")
        for j in range(len(class_labels)):
            print("{:6d}".format(conf_matrix[i, j]), end=" ")
        print("   True: {}".format(label))

    # Display the column labels
    print("\nPredicted:  ", end="")
    for label in class_labels:
        print("{:<12s}".format(label), end=" ")
    print()
    print("Accuracy:", accuracy)
    print()


Evaluating duration: 0.1 seconds
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.96      0.96       897
           1       0.96      0.99      0.98       852
           2       0.99      0.95      0.97       868

    accuracy                           0.97      2617
   macro avg       0.97      0.97      0.97      2617
weighted avg       0.97      0.97      0.97      2617

SVM Accuracy: 0.9675200611387085

Confusion Matrix:
Noise          862     23     12    True: Noise
Drone           10    842      0    True: Drone
Swarm Drone     32      8    828    True: Swarm Drone

Predicted:  Noise        Drone        Swarm Drone  
Accuracy: 0.9675200611387085

Evaluating duration: 0.2 seconds
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      1737
           1       0.97      0.99      0.98      1723
           2       0.99      0.96      0.98      1774

    